In [45]:
from exp.data_loader import *
from exp.nn import *

In [46]:
import math
import torch

In [47]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In Command mode press `h` to see all shortcuts

## data_loader

In [48]:
avaliable_urls()

MNIST_URL http://deeplearning.net/data/mnist/mnist.pkl


In [49]:
def normalize(x, m, s): return (x-m)/s
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [50]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [51]:
# num hidden
nh = 50

In [52]:
# simplified kaiming init / he init
#784, 50
w1 = torch.randn(m,nh)*math.sqrt(2./m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,10)*math.sqrt(2./nh)
b2 = torch.zeros(10)

In [54]:
#export
import math
import torch

In [55]:
x_train,y_train,x_valid,y_valid = get_data()

In [56]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = SoftmaxCrossEntropy()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def forward(self, x):
        for l in self.layers: x = l(x)
        return self.loss.softmax_forward(x)
    
    def backward(self, y_train):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
    
    def accuracy(self, pred, yb): return (torch.argmax(pred, dim=1)==yb).float().mean()

w1.g,b1.g,w2.g,b2.g = [None]*4

In [57]:
model = Model()
loss = model(x_train, y_train); loss

tensor(-0.0987)

In [58]:
model.backward(y_train)

In [59]:
pred = model.forward(x_train)
model.accuracy(pred, y_train)

tensor(0.1068)

In [70]:
bs = 10000    # batch size
lr = 0.01   # learning rate
epochs = 20# how many epochs to train for

In [71]:
for epoch in range(epochs):
    for i in range((n-1)//bs + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        loss = model(xb, yb)
        model.backward(yb)
        
        parameters = [w1,b1,w2,b2]
        for parameter in parameters:
            parameter-=lr*parameter.g
            
    pred = model.forward(x_train)
    print(model.accuracy(pred, y_train))
#     loss = model(x_train, y_train)
#     sm_pred = model.forward(x_train)
#     print()
#     print("accuracy", model.accuracy(x_train, y_train))
#     print("loss", loss)
        

tensor(0.4435)
tensor(0.4606)
tensor(0.4806)
tensor(0.4930)
tensor(0.5008)
tensor(0.5069)
tensor(0.5122)
tensor(0.5180)
tensor(0.5236)
tensor(0.5291)
tensor(0.5338)
tensor(0.5380)
tensor(0.5422)
tensor(0.5460)
tensor(0.5513)
tensor(0.5578)
tensor(0.5634)
tensor(0.5808)
tensor(0.5566)
tensor(0.5274)
